In [1]:
import numpy as np
from pomegranate import *

In [2]:
rain = Node(DiscreteDistribution({
    'none' : 0.7,
    'light' : 0.2,
    'heavy' : 0.1
}),name='rain')

maintenance = Node(ConditionalProbabilityTable([
    ['none','yes',0.4],
    ['none','no',0.6],
    ['light','yes',0.2],
    ['light','no',0.8],
    ['heavy','yes',0.1],
    ['heavy','no',0.9],
],[rain.distribution]),name='maintenance')

train = Node(ConditionalProbabilityTable([
    ['none','yes','on time',0.8],
    ['none','yes','delayed',0.2],
    ['none','no','on time',0.9],
    ['none','no','delayed',0.1],
    ['light','yes','on time',0.6],
    ['light','yes','delayed',0.4],
    ['light','no','on time',0.7],
    ['light','no','delayed',0.3],
    ['heavy','yes','on time',0.4],
    ['heavy','yes','delayed',0.6],
    ['heavy','no','on time',0.5],
    ['heavy','no','delayed',0.5],
],[rain.distribution,maintenance.distribution]),name='train')


appointment = Node(ConditionalProbabilityTable([
    ['on time','attend',0.9],
    ['on time','miss',0.1],
    ['delayed','attend',0.6],
    ['delayed','miss',0.4],
],[train.distribution]),name='appointment')

In [3]:
model = BayesianNetwork()

model.add_states(rain,maintenance,train,appointment)

In [4]:
model.add_edge(rain,maintenance)
model.add_edge(rain,train)
model.add_edge(maintenance,train)
model.add_edge(train,appointment)

In [5]:
model.bake()

In [6]:
prob = model.probability([['none','no','on time','attend']])
prob

0.34019999999999995

In [7]:
predictions = model.predict_proba({
    "train" : "delayed"
})

for node, prediction in zip(model.states, predictions):
    if isinstance(prediction, str):
        print(f"{node.name}: {prediction}")
    else:
        print(f"{node.name}")
        for value, probability in prediction.parameters[0].items():
            print(f" {value}: {probability:.4f}")

rain
 none: 0.4583
 light: 0.3069
 heavy: 0.2348
maintenance
 no: 0.6432
 yes: 0.3568
train: delayed
appointment
 miss: 0.4000
 attend: 0.6000
